# Phonon computations

This is a quick sketch how to run a simple phonon calculation using DFTK.

> **Preliminary implementation**
>
> Practical phonon computations have only seen rudimentary testing as of now.
> As of now we do not yet recommend relying on this feature for production
> calculations. We appreciate any issues, bug reports or PRs.

First we run an SCF calculation.

In [1]:
using AtomsBuilder
using DFTK
using PseudoPotentialData

pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
model  = model_DFT(bulk(:Si); pseudopotentials, functionals=LDA())
basis  = PlaneWaveBasis(model; Ecut=10, kgrid=[4, 4, 4])
scfres = self_consistent_field(basis, tol=1e-8);
nothing  # hide

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.916386041477                   -0.69    4.6    163ms
  2   -7.921195196612       -2.32       -1.52    1.0   58.6ms
  3   -7.921406854611       -3.67       -2.49    1.4   64.0ms
  4   -7.921440998772       -4.47       -2.85    2.5   80.7ms
  5   -7.921441723920       -6.14       -3.08    1.1   59.2ms
  6   -7.921442008785       -6.55       -4.56    1.0   82.0ms
  7   -7.921442021918       -7.88       -4.68    2.9   88.5ms
  8   -7.921442022135       -9.66       -5.26    1.0   58.3ms
  9   -7.921442022140      -11.36       -5.38    1.2   62.4ms
 10   -7.921442022144      -11.38       -6.04    1.0   59.3ms
 11   -7.921442022144      -12.55       -6.59    1.2   63.2ms
 12   -7.921442022144      -13.51       -7.30    1.5   70.4ms
 13   -7.921442022144      -14.45       -7.55    2.0   73.7ms
 14   -7.921442022144      -14.75       -8.56    1.0   59.3ms


Next we compute the phonon modes at the q-point `[1/4, 1/4, 1/4]`.

In [2]:
scfres = DFTK.unfold_bz(scfres)
phret_q0 = @time DFTK.phonon_modes(scfres; q=[0.25, 0.25, 0.25]);
nothing  # hide

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   2.51s  Non-interacting
   1        0       1        0.31     52.7   10.1s  
   2        0       2       -0.52     47.2   1.70s  
   3        0       3       -2.40     41.9   1.47s  
   4        0       4       -4.46     30.0   1.14s  
   5        0       5       -6.46     17.5   789ms  
   6        0       6       -8.92      4.0   438ms  
   7        1       1       -7.82     55.9   2.33s  Restart
   8        1       2       -8.97      5.7   1.26s  
                                      72.8   2.09s  Final orbitals
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      73.0   2.06s  Non-interacting
   1        0       1        0.31     52.7   1.91s  
   2        0       2       -0.52     47.2   1.67s  
   3

These are the final phonon frequencies:

In [3]:
phret_q0.frequencies

6-element Vector{Float64}:
 -0.0034981553611111195
 -0.0029774511624445876
 -0.0029774511620736
  0.004302230625230136
  0.004302230625579245
  0.00435320175472293